# **Esercitazione 2 - Regressione Lineare**

## Boston Housing dataset

Questo dataset contiene informazioni raccolte dal U.S. Census Service riguardanti le abitazioni nell'area di Boston, Massachusetts. È stato ottenuto dall'archivio StatLib (http://lib.stat.cmu.edu/datasets/boston) ed è stato ampiamente utilizzato in letteratura per fare benchmark di algoritmi. 

Il dataset contiene informazioni su 506 case, divise in 14 variabili.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import pandas as pd 
from sklearn.utils import shuffle
from pandas import read_csv

from sklearn.datasets import fetch_openml
import pandas as pd

# Scarica il Boston Housing Dataset da OpenML
boston = fetch_openml(name="Boston", version=1, as_frame=False)

# Estrai i dati (features) e il target (valore mediano delle abitazioni)
X = boston.data
y = boston.target

X, y = shuffle(X, y, random_state=0)
print(f"Features shape: {X.shape}, targets shape:  {y.shape}")

## `np.c_` in NumPy

L'oggetto `np.c_` in NumPy è una **scorciatoia** per concatenare array lungo il secondo asse (cioè, le colonne).

## Utilizzo
```python
np.c_[array1, array2, ...]


In [ ]:
import numpy as np

# Generate two random 2x3 matrices
matrice1 = np.random.rand(2, 3)
matrice2 = np.random.rand(2, 3)

# Concatenate the matrices along columns
risultato = np.c_[matrice1, matrice2]

print("Matrice 1:",matrice1.shape)

print("\nMatrice 2:",matrice2.shape)

print("\nMatrice concatenata:",risultato.shape)

**Divisione del dataset**

Il primo passaggio è quello di dividere i dati in train set, validation set e test set. Utilizza il 60% dei dati per il training set, il 20% per il validation e il restante 20% per il test set. Considerato che il nostro dataset possiede 506 osservazioni mi aspetto che:

- Il **training set** avrà 303 osservazioni.
- Il **validation set** avrà 101 osservazioni.
- Il **test set** avrà 101 osservazioni.

In reatà il test set avrà 102 osservazioni per via delle approssimazioni.



In [ ]:
# Divisione del dataset

train_porzione = 0.6  
val_porzione = 0.2  
test_porzione = 0.2

# svolgimento...
indici=X.shape[0]
train_indici=int(indici*train_porzione)
X_train=X[:train_indici]
y_train=y[:train_indici]
print("Train shape: ", X_train.shape)

val_indici=int(indici*val_porzione)
X_val=X[:val_indici]
y_val=y[:val_indici]
print("Validation shape: ", X_val.shape)

test_indici=indici-val_indici-train_indici
X_test=X[:test_indici]
y_test=y[:test_indici]
print("Validation shape: ", X_test.shape)

### **Esercizio 1: Costruisci una Pipeline di Regressione Lineare Standardizzata**

**Step 1:** Standardizza i dataset di addestramento, validazione e test. Usa `StandardScaler` di scikit-learn.  

**Step 2:** Aggiungi una feature costante (bias) ai dati concatenando una colonna di uno ad ogni dataset.  

**Step 3:** Implementa la soluzione in forma chiusa per l'addestramento di un modello di regressione lineare. 
 
**Step 4:** Valuta il modello calcolando il Mean Absolute Error (MAE) sui dataset di addestramento, validazione e test.


### **Guida**

1. **StandardScaler**:
   - Utilizza `StandardScaler` da `sklearn.preprocessing` per standardizzare i dati.
   - Il metodo `fit_transform` calcola la media e la varianza dei dati di addestramento e li scala di conseguenza.
   - Utilizza `transform` per standardizzare i dati di validazione e test utilizzando gli stessi parametri. Utilizziamo il metodo `transform` perchè non calcola i parametri di scaling (media e std). In questo modo ci assicuriamo che i dati di training e quelli di validation e test vengano scalati in modo uguale. Se usassimo `fit_transform` avremmo degli scaling diversi.

2. **Aggiunta di una Caratteristica Costante**:
   - Utilizza `np.c_` per concatenare una colonna di uno alle matrici delle caratteristiche. Questo è importante per includere il termine di intercetta nella regressione lineare.

3. **Soluzione in Forma Chiusa per la Regressione Lineare**:
   - La soluzione in forma chiusa è:

     $$\theta = (X^T X)^{-1} X^T y$$

   - Per calcolare la trasposta di una matrice possiamo utilizzare l' attributo `.T` di cui ogni array è dotato.

   - Utilizza `np.linalg.inv` di NumPy per l'inversione della matrice e l'operatore `@` per la moltiplicazione matriciale.
  
   - Puoi utilizzare l'operatore @ per eseguire l'operazione np.dot (`A @ B` è equivalente a `np.dot(A, B)`).

4. **Mean Absolute Error (MAE)**:
   - L'MAE si calcola come:

     $$\text{MAE} = \frac{1}{n} \sum_{i=1}^n |y_i - \hat{y}_i|$$

   - Utilizza `np.mean` e `np.abs` per calcolarlo.


In [ ]:
# Step 1 - Normalizzazione dei dati. Dobbiamo normalizzare le features 
# sia del training set, validation set e test set.
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()

# Utilizziamo il metodo .fit_transform() dello scaler per normalizzare le feature di training.
X_train_scaled=scaler.fit_transform(X_train)

# Per normalizzare le feature di validation e test utilizziamo il metodo .transform()
X_val_scaled=scaler.transform(X_val)
X_test_scaled=scaler.transform(X_test)

In [ ]:
# Step 2 - Aggiunta di una feature costante

# creiamo un vettore di 1 da aggiungere come feature costante. 
# ATTENZIONE: questo vettore deve avere le stesse righe del set a cui viene aggiunto. 
# Uno uguale per tutti non va bene

# svolgimento...
X_train_1=np.c_[np.ones((X_train_scaled.shape[0], 1)), X_train_scaled]
X_val_1=np.c_[np.ones((X_val_scaled.shape[0], 1)), X_val_scaled]
X_test_1=np.c_[np.ones((X_test_scaled.shape[0], 1)), X_test_scaled]

print("dimensioni")
print("train: ", X_train_1.shape)
print("validation: ", X_val_1.shape)
print("test: ", X_test_1.shape)

In [ ]:
# Step 3 - Applichiamo la formula matematica della regressione lineare

# ATTENZIONE: stiamo per effettuare operazioni tra matrici e vettori, 
# non si tratta di una semplice formula matematica, stiamo attenti a quali operatori utilizzare e quanto

# svolgimento...
X_trasposta=X_train_1.T

inverso=np.linalg.inv(X_trasposta@X_train_1)

parametri=(inverso@X_trasposta)@y_train

y_train_predizioni=X_train_1@parametri
y_val_predizioni=X_val_1@parametri
y_test_predizioni=X_test_1@parametri

print(y_train_predizioni.shape)
print(y_test_predizioni.shape)
print(y_val_predizioni.shape)

In [ ]:
# Step 4 - Calcolo MAE

# Calcoliamo l'errore medio assoluto (MAE) per il training set, validation set e test set.
# Utlizziamo la formula specificata nella guida.

# svolgimento...

totale_train=np.abs(y_train-y_train_predizioni)
MAE_train=np.mean(totale_train)
print(MAE_train)

totale_val=np.abs(y_val-y_val_predizioni)
MAE_val=np.mean(totale_val)
print(MAE_val)

totale_test=np.abs(y_test-y_test_predizioni)
MAE_test=np.mean(totale_test)
print(MAE_test)

### **Esercizio: Costruisci una pipeline di Regressione Lineare Standardizzata utilizzando `scikit-learn`** 

**Step 1 & 2:** Step 1 e 2 sono uguali a quanto fatto prima.

**Step 3:** Utilizza `LinearRegression()` di scikit-learn per addestrare un modello di regressione lineare.  

**Step 4:** Valuta il modello calcolando il Mean Absolute Error (MAE) sui dataset di addestramento, validazione e test, utilizzando `mean_absolute_error()` da `sklearn.metrics`.


## `LinearRegression` da Scikit-Learn

La classe `LinearRegression` in Scikit-Learn viene utilizzata per eseguire la **regressione lineare**, adattando un modello lineare al dataset.

## **Sintassi**
```python
from sklearn.linear_model import LinearRegression

model = LinearRegression()
# Dati di esempio
X = np.array([[1, 1], [1, 2], [2, 2], [2, 3]])
y = np.array([10, 15, 20, 25])

# Adatta il modello ai dati
model.fit(X, y)

# Predici nuovi valori
X_new = np.array([[3, 5], [5, 9]])
predictions = model.predict(X_new)


## `mean_absolute_error` da Scikit-Learn

La funzione `mean_absolute_error` calcola l'**errore assoluto medio** (MAE) tra i valori target reali e quelli predetti.

## **Sintassi**
```python
sklearn.metrics.mean_absolute_error(y_true, y_pred)


### **Guida**

1. **Istanziare e allenare un modello di regressione lineare**:
    
    - Istanziamo una classe `LinearRegression` per creare il modello.
    - Utilizziamo il metodo `.fit()` per allenare il modello con i dati di training.

2. **Effettuare predizioni con il modello**:

    - Utiliziamo il metodo `.predict()` del modello per effettuare le predizioni. Effettuiamo le predizioni per tutti i set che abbiamo (train, validation e test).

3. **Calcolo della MAE**: 

    - Calcolare MAE su tutti i set utilizzando la funzione `mean_abslute_error`


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

In [ ]:
# Step 1 - Istanziare e allenare il modello di regressione lineare.

# svolgimento...
model=LinearRegression()
model.fit(X_train, y_train)

In [ ]:
# Step 2 - Effettuare predizioni
print(type(X_train_1))

train_predition=model.predict(X_train)
val_predition=model.predict(X_val)
test_predition=model.predict(X_test)

In [ ]:
# Step 3 - Calcolo MAE

# svolgimento...
mae_train=mean_absolute_error(y_train, train_predition)
mae_val=mean_absolute_error(y_val, val_predition)
mae_test=mean_absolute_error(y_test, test_predition)

print(mae_train)
print(mae_val)
print(mae_test)

### **Esercizio: Crea una funzione che esegua una pipeline di Regressione Lineare**

La funzione deve richiedere un parametro `hyperparams` per gestire i diversi casi. 

`hyperparams` deve essere un dizionario contenente diverse chiavi, in base al valore di queste chiavi devono essere eseguiti (oppure no) diversi pezzi di codice. 

In questo esercizio la chiave da utilizzare sarà `hyperparams['data_standardize']`. Se il valore di questa chiave sarà **True** allora eseguire la standardizzazione con `scikit-learn`, se invece è **False** non verrà eseguita alcuna standardizzazione.

**Step 1:** Controllare se eseguire o no la standardizzazione.

* **Step 1.1:** Scrivere il codice per eseguire la standardizzazione.

**Step 2:** Utilizza `np.c_` per concatenare una colonna di uno alle matrici delle caratteristiche.

**Step 3:** Applichiamo la formula matematica della regressione lineare.

**Step 4:** Calcolo MAE utilizzando la formula (NON con `scikit-learn`).

La funzione deve ritornare i valori della MAE.

Dopo aver testato i risultati con `hyperparams['data_standardize']` = **True**, provare anche i risultati ottenuti se `hyperparams['data_standardize']` = **False**.

In [ ]:
# svolgimento...

def pipeline(X_train, y_train, X_val, y_val, hyperparams):

    X_train = np.array(X_train, dtype=float)
    y_train = np.array(y_train, dtype=float)
    X_val = np.array(X_val, dtype=float)
    y_val = np.array(y_val, dtype=float)
    
    # Step 1 - Controllo se è richiesta la standardizzazione dei dati
    if hyperparams['data_standardize']:
        
        # Step 1.1 - Scrivere il codice per standardizzare i dati
        scaler = StandardScaler()
        X_train_scaled=scaler.fit_transform(X_train)
        X_val_scaled=scaler.transform(X_val)
    
    # Step 2 - Concatenare una colonna di uno alla matrice delle features
    X_train_1=np.c_[np.ones((X_train_scaled.shape[0], 1)), X_train_scaled]
    X_val_1 = np.c_[np.ones((X_val_scaled.shape[0], 1)), X_val_scaled]
        
    # Step 3 - Applicare formula della regressione lineare e calcolare predizioni
    X_trasposta = X_train_1.T  
    inversa = np.linalg.inv(X_trasposta @ X_train_1) 
    parametro=(inversa @ X_trasposta) @ y_train
 
    y_train_pred = X_train_1 @ parametro
    y_val_pred = X_val_1 @ parametro   
    
    # Step 4 - Calcolare MAE     
    MAE_val=np.mean(np.abs(y_val-y_val_pred))
    MAE_train=np.mean(np.abs(y_train-y_train_pred))
    return(MAE_val , MAE_train)

In [ ]:
hyperparams = {'data_standardize': True}

train_fraction = 0.8
validation_fraction = 0.2

num_train = int(train_fraction * X.shape[0])

X_train = X[:num_train]
y_train = y[:num_train]

X_validation = X[num_train:]
y_validation = y[num_train:]

# Chiamare la funzione pipeline e stampare i risultati della MAE

# svolgimento...
mae_val, mae_train =pipeline(X_train, y_train, X_validation, y_validation, hyperparams)
print(f'Mean Absolute Error (MAE) sul training:', mae_train)
print(f'Mean Absolute Error (MAE) sulla validazione:', mae_val)

### **Esercizio: Implementare alla funzione `pipeline` la possibilità di usare PCA**

Modifichiamo la funzione `pipeline` in modo da gestire anche la possibilità di effettuare la PCA. Dunque aggiungiamo al dizionario `hyperparams` la chiave `use_pca`. 

Se `hyperparams['use_pca']` = **True** verrà eseguita la PCA. 

Se `hyperparams['use_pca']` = **False** non verrà eseguita la PCA.

La gestione della standardizzazione deve essere mantenuta come prima.

In [ ]:
# svolgimento...
from sklearn.decomposition import PCA

def pipeline(X_train, y_train, X_val, y_val, hyperparams):

    X_train = np.array(X_train, dtype=float)
    y_train = np.array(y_train, dtype=float)
    X_val = np.array(X_val, dtype=float)
    y_val = np.array(y_val, dtype=float)

    # Step 1 - Controllo se è richista la PCA
    if hyperparams['use_pca']:
        
        # Step 1.1 - Scrivere il codice per applicare PCA
        scaler = StandardScaler()
        X_train_scaled=scaler.fit_transform(X_train)
        X_val_scaled=scaler.transform(X_val)
        pca=PCA()
        X_train_pca=pca.fit_transform(X_train_scaled)
        X_val_pca=pca.transform(X_val_scaled)
        
    # Step 2 - Controllo se è richiesta la standardizzazione dei dati
    if hyperparams['data_standardize']:
        
        # Step 2.1 - Scrivere il codice per standardizzare i dati 
        scaler = StandardScaler()
        X_train_scaled=scaler.fit_transform(X_train)
        X_val_scaled=scaler.transform(X_val)

    # Step 3 - Concatenare una colonna di uno alla matrice delle features
    X_train_1=np.c_[np.ones((X_train.shape[0], 1)), X_train_scaled]
    X_val_1= np.c_[np.ones((X_val.shape[0], 1)), X_val_scaled]
    
    # Step 4 - Applicare formula della regressione lineare e calcolare predizioni
    X_trasposta = X_train_1.T  
    inversa = np.linalg.inv(X_trasposta @ X_train_1 ) 
    parametri=(inversa @ X_trasposta) @ y_train
 
    y_train_pred = X_train_1 @ parametri
    y_val_pred = X_val_1 @ parametri

    # Step 5 - Calcolare MAE 
    MAE_val=np.mean(np.abs(y_val-y_val_pred))
    MAE_train=np.mean(np.abs(y_train-y_train_pred))
    return MAE_val, MAE_train

In [ ]:
hyperparams = {'data_standardize': True, 'use_pca': True}
train_fraction = 0.8
validation_fraction = 0.2

num_train = int(train_fraction * X.shape[0])

X_train = X[:num_train]
y_train = y[:num_train]

X_validation = X[num_train:]
y_validation = y[num_train:]

# Chiamare la funzione pipeline e stampare i risultati della MAE al variare dell' utilizzo della PCA.

# svolgimento...
mae_val, mae_train =pipeline(X_train, y_train, X_validation, y_validation, hyperparams)
print(f'Mean Absolute Error (MAE) sul training:', mae_train)
print(f'Mean Absolute Error (MAE) sulla validazione:', mae_val)